In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s  0:00:00
Using cached torch-2.9.0-cp310-cp310-manylinux_2_28_x86_64.whl (899.8 MB)
Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl (594.3 MB)
Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (10.2 MB)
Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl (88.0 MB)
Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (954 kB)
Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl (706.8 MB)
Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (193.1 MB)
Using cached nvidia_cufile_cu12-1.13.1.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (1.2 MB)
Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl (63.6 MB)
Using cached nvidia_cusolver_cu12-11.7.3.90-py

In [ ]:
import gc
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from ultralytics import YOLO
from tensorflow.keras.models import load_model


## Segmentación en frame completo + filtrado con ROI YOLO

**Proceso del código:**

1) Obtiene una **región de interés (ROI) global** recorriendo todo el videocon **YOLO**.  
2) Para **cada frame completo del video**, aplica **UNet** para generar la segmentación.  
3) Filtra la segmentación de UNet usando la ROI de YOLO, **eliminando la segmentación fuera de la zona relevante**.  
4) Dibuja los contornos de la segmentación sobre el **video original completo**, manteniendo la resolución original.


In [ ]:
#############################################
#   1) FUNCIÓN PARA OBTENER LA MÁSCARA UNET #
#############################################
def get_unet_mask(image, unet_model):
    """
    Ajustado para un modelo que espera (None, 128, 128, 3).
    image: Frame con 3 canales (BGR) de OpenCV.
    """

    # 1) Convertimos a float32
    img_float = image.astype(np.float32)

    # 2) Redimensionamos a 128x128
    #    ¡Ojo! Ajusta a las dimensiones exactas que usa tu modelo: 128 x 128
    img_resized = cv2.resize(img_float, (128, 128))

    # 3) Normalizamos si tu modelo se entrenó en [0,1]
    img_resized /= 255.0

    # 4) Expandimos dimensiones para el batch => (1, 128, 128, 3)
    img_input = np.expand_dims(img_resized, axis=0)

    # 5) Inferencia con el modelo
    seg_pred = unet_model.predict(img_input)  
    #    seg_pred podría tener forma (1, 128, 128, 1) ó (1, 128, 128, X)

    # 6) Quitamos la dimensión de batch => (128, 128, canales_salida)
    mask = np.squeeze(seg_pred)

    # 7) Si el modelo produce más de un canal de salida (p.ej. 2 clases),
    #    escoge el canal que te interese. Aquí, por simplicidad, canal 0:
    if mask.ndim == 3 and mask.shape[2] > 1:
        mask = mask[..., 0]

    # 8) Binarizamos => 0/1
    mask = np.round(mask)

    # 9) Redimensionar la máscara de vuelta al tamaño original de la imagen
    orig_h, orig_w = image.shape[:2]
    mask = cv2.resize(mask, (orig_w, orig_h))

    # 10) Convertir a booleano (opcional, si quieres manipularlo como máscara)
    mask = mask.astype(bool)

    return mask


###############################################
#   2) FUNCIÓN PARA OBTENER ROI CON YOLO     #
###############################################
def get_max_yolo_roi(video_path, yolo_model, margin=15):
    """
    Recorre todo el video y obtiene la ROI que cubre todas las detecciones YOLO.
    Devuelve las coordenadas (x1, y1, x2, y2) de la ROI con margen.
    """
    cap = cv2.VideoCapture(video_path)
    min_x1, min_y1 = float('inf'), float('inf')
    max_x2, max_y2 = 0, 0

    max_roi_with_margin = None  # Por si no se detecta nada
    max_img = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Inferencia con YOLO
        results = yolo_model(frame)
        # YOLO >= 8.0: results[0].boxes / YOLO <8.0: results.xyxy[0]
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().int().numpy()

                # Ajustar coordenadas a los límites del frame
                x1 = max(0, min(x1, frame.shape[1] - 1))
                y1 = max(0, min(y1, frame.shape[0] - 1))
                x2 = max(0, min(x2, frame.shape[1] - 1))
                y2 = max(0, min(y2, frame.shape[0] - 1))

                # Actualizar min y max para la ROI
                min_x1 = min(min_x1, x1)
                min_y1 = min(min_y1, y1)
                max_x2 = max(max_x2, x2)
                max_y2 = max(max_y2, y2)

                max_img = frame.copy()

    cap.release()

    # Agregar márgenes asegurándonos de no salirnos de la imagen
    if max_img is not None:
        min_x1 = max(0, min_x1 - margin)
        min_y1 = max(0, min_y1 - margin)
        max_x2 = min(max_img.shape[1] - 1, max_x2 + margin)
        max_y2 = min(max_img.shape[0] - 1, max_y2 + margin)
        max_roi_with_margin = (min_x1, min_y1, max_x2, max_y2)
    else:
        print("No se detectó ninguna ROI en las imágenes del video.")

    return max_roi_with_margin

###############################################
#   3) FUNCIÓN PARA FILTRAR LA MÁSCARA UNET  #
###############################################
def filter_unet_mask_with_yolo(unet_mask, roi):
    """
    Filtra la máscara de UNet para mantener solo las áreas dentro de la ROI (x1, y1, x2, y2).
    """
    x1, y1, x2, y2 = roi
    filtered_mask = np.zeros_like(unet_mask)
    filtered_mask[y1:y2, x1:x2] = unet_mask[y1:y2, x1:x2]
    return filtered_mask

##################################################
#   4) PROCESAR VIDEO, OBTENER SEGMENTACIÓN      #
#      Y DIBUJAR CONTORNOS                       #
##################################################
def process_and_create_segmented_video_with_contours(
    video_path,
    yolo_model,
    unet_model,
    output_video_path,
    margin=10
):
    """
    Lee frame a frame el video:
      - Obtiene la ROI global con YOLO (get_max_yolo_roi).
      - Para cada frame, aplica get_unet_mask y filtra con filter_unet_mask_with_yolo.
      - Dibuja contornos y guarda el nuevo video en output_video_path.
    """
    # 1) Calcular ROI con YOLO
    roi = get_max_yolo_roi(video_path, yolo_model, margin)

    # Abrimos el video
    cap = cv2.VideoCapture(video_path)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Configurar writer de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while cap.isOpened():
        ret, frame_video = cap.read()
        if not ret:
            break

        # 2) Obtener la máscara de SERUNet
        unet_mask = get_unet_mask(frame_video, unet_model)

        # 3) Filtrar la máscara con la ROI
        if roi is not None:
            filtered_mask = filter_unet_mask_with_yolo(unet_mask, roi)
        else:
            # Si no hay ROI detectada, la máscara queda igual
            filtered_mask = unet_mask

        # 4) Convertir a uint8 y buscar contornos
        filtered_mask_resized = filtered_mask.astype(np.uint8)
        contours, _ = cv2.findContours(filtered_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 5) Dibujar los contornos en el frame original
        frame_with_contours = frame_video.copy()
        cv2.drawContours(frame_with_contours, contours, -1, (0, 255, 0), 2)

        # 6) Agregar frame al video de salida
        output_video.write(frame_with_contours)

        frame_count += 1
        # Liberar memoria ocasionalmente
        if frame_count % 100 == 0:
            gc.collect()
            tf.keras.backend.clear_session()

    cap.release()
    output_video.release()

    print(f"Video con segmentación y contornos creado en: {output_video_path}")




In [ ]:
video_path = "/home/vplab-2024/Desktop/VoiceLab/segmentation_glottis/datasets/videos_VPLab/FN001.avi"
output_video_path = "/home/vplab-2024/Desktop/VoiceLab/segmentation_glottis/models/videos/FN001_s3ar-yolo_roi1_1.mp4"
yolo_model_path = "//home/vplab-2024/Desktop/VoiceLab/segmentation_glottis/models/YOLO/YOLOV8/best_yolov8n-seg-1cls.pt"
unet_model_path = "/home/vplab-2024/Desktop/VoiceLab/segmentation_glottis/models/UNets/S3AR-UNet/s3ar_unet/model/SeARUNet-2/SeARUNet-2.h5"

yolo_model = YOLO(yolo_model_path)
unet_model = load_model(unet_model_path, compile=False)

process_and_create_segmented_video_with_contours(
    video_path=video_path,
    yolo_model=yolo_model,
    unet_model=unet_model,
    output_video_path=output_video_path,
    margin=10
)


## Segmentación directa sobre ROI recortada con YOLO

**Proceso del código:**

1) Obtiene una **región de interés (ROI) global** recorriendo todo el video con **YOLO**.  
2) **Recorta cada frame** del video usando la ROI obtenida con YOLO.  
3) Aplica **UNet únicamente sobre la imagen recortada**.  
4) Dibuja los contornos de la segmentación sobre el **video recortado**, que es el video de salida final.


In [ ]:
def pipeline_s3arunet(
    input_video_path,
    s3arunet_path,
    yolo_model_path,
    output_video_path,
    margin=15,
    model_input_size=(128, 128)
):
    """
    Procesa un video usando YOLO para hallar una ROI global y luego un modelo S3ARUnet
    para segmentar la parte recortada del video. El resultado se guarda como video con
    contornos dibujados.

    Parámetros:
    -----------
    - input_video_path : str
        Ruta del video de entrada.
    - s3arunet_path : str
        Ruta del modelo S3ARUnet (archivo .h5).
    - yolo_model_path : str
        Ruta del modelo YOLO (ej. un .pt entrenado).
    - output_video_path : str
        Ruta de salida para el video segmentado.
    - margin : int, opcional (default=15)
        Margen de píxeles alrededor de la ROI detectada.
    - model_input_size : tuple(int, int), opcional (default=(128,128))
        Tamaño (ancho, alto) que necesita el modelo S3ARUnet.
        Ajustar a (width, height) según tu entrenamiento.
    """

    #################################################
    # 1) Función auxiliar: Dibujar contornos en BGR #
    #################################################
    def draw_contours_on_image(mask, img_bgr, contour_color=(0, 255, 0), contour_thickness=3):
        """
        Encuentra contornos en 'mask' (binaria) y los dibuja en la imagen 'img_bgr'.
        """
        contours, _ = cv2.findContours(
            mask.astype(np.uint8),
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE
        )
        # Asegurar que la imagen sea BGR
        if len(img_bgr.shape) == 2 or img_bgr.shape[2] == 1:
            img_with_contours = cv2.cvtColor(img_bgr, cv2.COLOR_GRAY2BGR)
        else:
            img_with_contours = img_bgr.copy()

        cv2.drawContours(
            img_with_contours,
            contours,
            -1,
            contour_color,
            contour_thickness
        )
        return img_with_contours

    #################################################
    # 2) Función auxiliar: Obtener ROI global YOLO  #
    #################################################
    def get_max_yolo_roi(video_path, yolo_model, margin=15):
        """
        Recorre todo el video para determinar la bounding box global
        que cubra todas las detecciones YOLO.
        """
        cap = cv2.VideoCapture(video_path)
        min_x1, min_y1 = float('inf'), float('inf')
        max_x2, max_y2 = 0, 0
        max_img = None

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = yolo_model(frame)
            if results[0].boxes is not None and len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    xA, yA, xB, yB = box.xyxy[0].cpu().int().numpy()

                    # Limitar a los bordes de la imagen
                    xA = max(0, min(xA, frame.shape[1] - 1))
                    yA = max(0, min(yA, frame.shape[0] - 1))
                    xB = max(0, min(xB, frame.shape[1] - 1))
                    yB = max(0, min(yB, frame.shape[0] - 1))

                    # Actualizar min/max
                    min_x1 = min(min_x1, xA)
                    min_y1 = min(min_y1, yA)
                    max_x2 = max(max_x2, xB)
                    max_y2 = max(max_y2, yB)

                    max_img = frame.copy()

        cap.release()

        if max_img is not None:
            # Agregar margen
            min_x1 = max(0, min_x1 - margin)
            min_y1 = max(0, min_y1 - margin)
            max_x2 = min(max_img.shape[1] - 1, max_x2 + margin)
            max_y2 = min(max_img.shape[0] - 1, max_y2 + margin)
            return (min_x1, min_y1, max_x2, max_y2)
        else:
            print("No se detectó ninguna ROI en el video.")
            return (0, 0, 0, 0)

    ###################################################################
    # 3) Función auxiliar: Redimensionar y normalizar para el modelo  #
    ###################################################################
    def resize_frame_to_model(cropped_frame, target_size):
        """
        Redimensiona (y normaliza a [0..1]) la imagen recortada para adecuarla
        a la entrada del modelo S3ARUnet.
        """
        width, height = target_size  # target_size = (128, 128) => (W, H)
        resized_frame = cv2.resize(cropped_frame, (width, height))
        # Convertir a float en [0..1]
        resized_frame = resized_frame.astype(np.float32) / 255.0
        return np.expand_dims(resized_frame, axis=0)  # (1, H, W, 3)

    #################################################
    #   4) Comienza la lógica principal del pipeline
    #################################################

    # a) Cargamos los modelos
    print("Cargando modelo YOLO...")
    yolo_model = YOLO(yolo_model_path)

    print("Cargando modelo S3ARUnet...")
    s3ar_unet = load_model(
        s3arunet_path,
        compile=False,
        custom_objects={"InstanceNormalization": tfa.layers.InstanceNormalization}
    )

    # b) Determinamos la ROI global con YOLO
    x1, y1, x2, y2 = get_max_yolo_roi(input_video_path, yolo_model, margin=margin)
    print(f"ROI global con margen: (x1={x1}, y1={y1}, x2={x2}, y2={y2})")

    # c) Abrimos el video y configuramos el writer de salida
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = None
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    frame_count = 0

    print("Procesando frames...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # d) Recortar el frame con la ROI
        cropped_frame = frame[y1:y2, x1:x2]
        if cropped_frame.size == 0:
            # ROI inválida o no detectada
            # Se puede usar frame completo como fallback
            cropped_frame = frame

        # e) Redimensionar al tamaño que espera el modelo
        input_batch = resize_frame_to_model(cropped_frame, model_input_size)

        # f) Inferencia con S3ARUnet
        seg_pred = s3ar_unet.predict(input_batch)
        # seg_pred => (1, H, W, 1) o similar
        seg_pred = np.squeeze(seg_pred)  # => (H, W) o (H, W, C)

        # Si el modelo tiene varios canales de salida y solo quieres el primero:
        # if seg_pred.ndim == 3 and seg_pred.shape[-1] > 1:
        #     seg_pred = seg_pred[..., 0]

        # g) Binarizar
        mask_binary = (seg_pred > 0.5).astype(np.uint8)

        # h) Redimensionar la máscara a las dimensiones del recorte original
        h_c, w_c = cropped_frame.shape[:2]
        mask_restored = cv2.resize(
            mask_binary,
            (w_c, h_c),  # ancho, alto
            interpolation=cv2.INTER_NEAREST
        )

        # i) Dibujar contornos sobre el recorte
        img_with_contours = draw_contours_on_image(
            mask=mask_restored,
            img_bgr=cropped_frame,
            contour_color=(0, 255, 0),
            contour_thickness=2
        )

        # j) Inicializar el writer de video si no se ha hecho
        if out is None:
            h_out, w_out = img_with_contours.shape[:2]
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (w_out, h_out))

        # k) Escribir el frame en el video
        out.write(img_with_contours)
        frame_count += 1

    # l) Liberar recursos
    cap.release()
    if out is not None:
        out.release()

    print(f"Proceso finalizado. Se creó el video: {output_video_path}")


In [ ]:
pipeline_s3arunet(
    input_video_path,
    s3arunet_path,
    yolo_model_path,
    output_video_path,
    margin=15,
    model_input_size=(128, 128)
):